http://math.hws.edu/xJava/GA/

In [10]:
import numpy as np
import operator

# Create And fill map with food 

In [4]:
wm = 30
hm = 30
def create_map() : 
    n_food = 100
    Map = np.zeros((wm,hm))
    #Setup walls
    Map[0,:]=1
    Map[:,0]=1
    Map[-1,:] =1
    Map[:,-1] =1
    # Put food 
    for i in range(1,n_food) : Map[np.random.randint(1,hm-1),np.random.randint(1,wm-1)]=2    
    return Map

# Create Eaters

In [5]:
n_state = 16 #0...15
n_poss_choice = 4 #0...3 0: forward 1: backward 2: turn right 3: turn left 
n_if = 4 # 0...3
len_chromosome = 64

In [21]:
class Eater : 
    state = 0
    score = 0
    history_moves = []
    
    def __init__(self):
        self.position = np.random.randint(1,wm-1,(1,2))[0]
        self.direction = np.random.randint(0,4,1) # 0 : up 1: right 2: down 3 : left
        self.initialize_chromosome()
        
    def get_legacy(self,old_chromosome):
        self.chromosome = old_chromosome
    
    def initialize_chromosome(self) :
        ''' Initialize the chromosome with random actions and states
        '''
        self.chromosome = np.empty((len_chromosome,2),dtype=int)
        self.chromosome[:,0]=np.random.randint(0,n_if,(len_chromosome,)) # Actions
        self.chromosome[:,1]=np.random.randint(0,n_state,(len_chromosome,)) # States
    
    def do_one_step(self) : 
        [action,self.state] = self.get_decision()
        self.act(action)
    
    def get_decision(self) :
        '''
            Return new action and state from the chromosome given a state and what is the object in front.
            i_F ; In front
            state : previous state 
        '''
        if self.direction == 0 : view = [-1,0]
        if self.direction == 1 : view = [0,1]
        if self.direction == 2 : view = [1,0]
        if self.direction == 3 : view = [0,-1]
        view_pos = np.array(self.position)+np.array(view)
        i_F = Map[view_pos[0],view_pos[1]]
        [action,state] = self.chromosome[int(i_F*n_state+self.state),:]
        return(action,state)
    
    def act(self,action) : 
        '''
        Realize the action 
        action: #0...3 0: forward 1: backward 2: turn right 3: turn left 
        '''
        motion = [0,0]
        if action == 0 : 
            if self.direction == 0 : motion = [-1,0]
            if self.direction == 1 : motion = [0,1]
            if self.direction == 2 : motion = [1,0]
            if self.direction == 3 : motion = [0,-1]
        if action == 1 : 
            if self.direction == 0 : motion = [1,0]
            if self.direction == 1 : motion = [0,-1]
            if self.direction == 2 : motion = [-1,0]
            if self.direction == 3 : motion = [0,1]
        if action == 2 : 
            self.direction +=1 
            if self.direction == 4 : self.direction =0
        if action == 3 :
            self.direction -=1 
            if self.direction == -1 : self.direction =3
        self.position = self.move(motion)
        self.eat()
        
        
    def move(self,motion):
        n_position = np.array(self.position)+np.array(motion)
        if Map[n_position[0],n_position[1]] != 1 and Map[n_position[0],n_position[1]] != 3 : 
            self.history_moves.append(n_position)
            return n_position
        else : return self.position
    
    def eat(self) : 
        if Map[self.position[0],self.position[1]] == 2 : 
            self.score +=1
            Map[self.position[0],self.position[1]] = 0
            Map[np.random.randint(1,hm-1),np.random.randint(1,wm-1)]=2

    def mate(self,eater2) : 
        '''
            Mate with another eater : their chromosomes crossover
        '''
        temp = self.chromosome.copy()
        idx = np.random.randint(0,64)
        temp[idx:,:] = eater2.chromosome[idx:,:]
        return temp

In [52]:
def mating(eaters) : 
    children = list()
    t =sorted(eaters,key=operator.attrgetter('score'),reverse=True)
    selection = t[0:int(round(n_eaters*p_selection))]
    while len(children) < max_children :
        i = np.random.randint(0,len(selection))
        j = np.random.randint(0,len(selection))
        if i!=j : 
            c = selection[i].mate(selection[j])
            child = Eater()
            child.get_legacy(c)
            children.append(child)
    return children

In [53]:
def mutate(eaters) : 
    for i in range(0,n_mutation) : 
        eaters[np.random.randint(0,n_eaters)].chromosome[np.random.randint(0,len_chromosome),:] = [np.random.randint(0,n_if),np.random.randint(0,n_state)]

In [22]:
n_eaters = 100
p_selection=0.33
max_children = 100
n_mutation = 1
Map=create_map()
eaters = [Eater() for e in range(1,n_eaters) ]

for i in range(0,1) :
    for i in range(1,100) : 
        [a.do_one_step() for a in eaters]
    print('Mean Score : {}'.format(int(np.mean([a.score for a in eaters]))))
    #eaters = mating(eaters)
    #mutate(eaters)

Mean Score : 1


In [37]:
set([tuple(h) for h in eaters[0].history_moves])

{(1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 17),
 (1, 18),
 (1, 19),
 (1, 20),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 13),
 (2, 14),
 (2, 15),
 (2, 18),
 (2, 19),
 (2, 20),
 (2, 21),
 (3, 1),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 10),
 (3, 12),
 (3, 13),
 (3, 14),
 (3, 15),
 (3, 18),
 (3, 20),
 (3, 21),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (4, 10),
 (4, 11),
 (4, 12),
 (4, 13),
 (4, 14),
 (4, 15),
 (4, 17),
 (4, 18),
 (4, 19),
 (4, 20),
 (4, 21),
 (4, 22),
 (4, 25),
 (4, 27),
 (4, 28),
 (5, 1),
 (5, 5),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (5, 10),
 (5, 13),
 (5, 14),
 (5, 15),
 (5, 17),
 (5, 20),
 (5, 21),
 (5, 22),
 (5, 23),
 (5, 25),
 (5, 26),
 (5, 27),
 (5, 28),
 (6, 1),
 (6, 2),
 (6, 5),
 (6, 6),
 (6, 7),
 (6, 8),
 (6, 9),
 (6, 10),
 (6, 11),
 (6, 12),
 (6, 13),
 (6, 14),
 (6, 15),
 (6, 16),
 (6